In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import gzip
import gensim 
import logging
import tensorflow as tf
import nltk
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
print(tf.__version__)


2.1.0


In [2]:
import csv
def read(input_file):
  with open(input_file, newline='') as f:
      for i, line in enumerate(f):
       yield gensim.utils.simple_preprocess (line, min_len=4, max_len=30)
   

In [3]:
documents=list(read('Chemdatatest_final.csv'))


In [4]:
all_stopwords = gensim.parsing.preprocessing.STOPWORDS
for i in range (len(documents)):
    for j in range(len(documents[i])):
        if documents[i][j] in all_stopwords:
            documents[i][j] = " "
            
        

In [5]:
def remove_values_from_list(the_list, val):
   return [value for value in the_list if value != val]


In [6]:
for i in range (len(documents)):
    documents[i] = remove_values_from_list(documents[i]," ")

In [7]:
count1=0
for i in range(len(documents)):
  count1=count1+documents[i].count("acrylic")
count1

16

In [8]:
length=[]
for i in range(len(documents)):
  length.append(len(documents[i]))

In [9]:
len(length)

37414

In [10]:
for i in range(len(documents)):
  for j in range(length[i]-1):
    documents[i].append(documents[i][j]+" "+documents[i][j+1])

#two words

In [11]:
#for i in range(len(documents)):
 # for j in range(length[i]-2):
  #  documents[i].append(documents[i][j]+" "+documents[i][j+1]+" "+documents[i][j+2])

#three words

In [12]:

model = gensim.models.Word2Vec (documents, size=120, window=15, min_count=1, workers=10, sg=1, iter=10, sorted_vocab=1)
model.train(documents,total_examples=len(documents),epochs=20)

2020-08-23 01:41:10,750 : INFO : collecting all words and their counts
2020-08-23 01:41:10,751 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-08-23 01:41:10,813 : INFO : PROGRESS: at sentence #10000, processed 195124 words, keeping 71820 word types
2020-08-23 01:41:10,912 : INFO : PROGRESS: at sentence #20000, processed 453145 words, keeping 185229 word types
2020-08-23 01:41:10,983 : INFO : PROGRESS: at sentence #30000, processed 658728 words, keeping 259226 word types
2020-08-23 01:41:11,062 : INFO : collected 336600 word types from a corpus of 881455 raw words and 37414 sentences
2020-08-23 01:41:11,063 : INFO : Loading a fresh vocabulary
2020-08-23 01:41:12,430 : INFO : effective_min_count=1 retains 336600 unique words (100% of original 336600, drops 0)
2020-08-23 01:41:12,432 : INFO : effective_min_count=1 leaves 881455 word corpus (100% of original 881455, drops 0)
2020-08-23 01:41:13,412 : INFO : deleting the raw counts dictionary of 336600 items

(17625950, 17629100)

In [13]:
import pickle
from azureml.core import Experiment
from azureml.core import Model
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

2020-08-23 01:44:46,320 : WARNING : Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (portalocker 2.0.0 (/anaconda/envs/azureml_py36/lib/python3.6/site-packages), Requirement.parse('portalocker~=1.0'), {'msal-extensions'}).


In [14]:
import azureml.core
from azureml.core import Workspace
ws = Workspace.from_config()
experiment = Experiment(workspace = ws, name = "chem_test")
run = experiment.start_logging()

2020-08-23 01:44:46,585 : INFO : Found the config file in: /config.json
2020-08-23 01:44:46,807 : INFO : b4235079-7b2f-4a00-993b-69d4096862fb - CacheDriver:Cached token is expired at 2020-08-22 21:01:13.584433.  Refreshing
2020-08-23 01:44:46,808 : INFO : b4235079-7b2f-4a00-993b-69d4096862fb - TokenRequest:Getting a new token from a refresh token
2020-08-23 01:44:47,158 : INFO : b4235079-7b2f-4a00-993b-69d4096862fb - CacheDriver:Returning token refreshed after expiry.
2020-08-23 01:44:47,414 : INFO : Created a worker pool for first use


In [15]:
model_file='chem_test.pkl'
joblib.dump(value=model, filename=model_file)
run.upload_file(name = 'outputs/' + model_file, path_or_stream = './' + model_file)

In [16]:
run.complete()


In [17]:
run.register_model(model_path='outputs/chem_test.pkl', model_name='chem_test')

Model(workspace=Workspace.create(name='HT6MLCHEM', subscription_id='8a658383-aed6-4735-9cb7-7bcab07d8fca', resource_group='HT6'), name=chem_test, id=chem_test:9, version=9, tags={}, properties={})

In [18]:
from azureml.core import Model
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    print('\n')

chem_test version: 9


chem_sim version: 3


chem_sim version: 2


chem_sim version: 1


chem_test version: 8


chem_test version: 7


chem_test version: 6


chem_test version: 5


chem_test version: 4


chem_test version: 3


diabetes_model version: 2


chem_test version: 2


chem_test version: 1


diabetes_model version: 1




In [19]:
model = ws.models['chem_test']
print(model.name, 'version', model.version)

chem_test version 9


In [20]:
import os

folder_name = 'chem_service'

# Create a folder for the web service files
experiment_folder = './' + folder_name
os.makedirs(folder_name, exist_ok=True)

print(folder_name, 'folder created.')

chem_service folder created.


In [21]:
%%writefile $folder_name/chem_predictions.py
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 
import json
import joblib
import numpy as np
from azureml.core.model import Model
mem=set()
keyword_mem=set()
# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = Model.get_model_path('chem_test')
    model = joblib.load(model_path)
def post_processing(w1, word):
    word_list=[]
    for t in range(len(word)):
        word_list.append(list(word[t]))
    ps = PorterStemmer() 
    if " " in w1:
        #if (ps.stem(w1.split()[0]) in mem) or (ps.stem(w1.split()[1]) in mem):
          #  return
        #else:
        mem.add(ps.stem(w1.split()[0]))
        mem.add(ps.stem(w1.split()[1]))
    else:    
       #if ps.stem(w1) in mem:
         #  return
       # else:
        mem.add(ps.stem(w1))
    for i in range(len(word_list)):
        if " " in word_list[i][0]:
            phrase=list(word_list[i][0].split()) 
            # print("The phrase is " + str(phrase))
            # flag = True
            # print("Duplicate "+phrase[0] +" "+ phrase[1])
            if phrase[0] == phrase[1]:
                word_list[i][0]=word_list[i][0].split()[0]
                del phrase[1]
                if ps.stem(phrase[0]) in mem:
                    word_list[i][0]="DELETE"
            else:
                count = 0
                for j in range (len(phrase)):
                    # print(str(j)+" "+ str(phrase) + " "+phrase[j])
                    ww=ps.stem(phrase[j])
                    if ww in mem:
                        count = count + 1
                        if(count == 1):
                            word_list[i][0] = word_list[i][0].split()[1-j]
                        else:
                            word_list[i][0]="DELETE"
                    # flag = False 
                    else:
                        mem.add(ww)
        else:
            w = ps.stem(word_list[i][0])
            if w in mem:
                word_list[i][0]="DELETE"
            else:
                mem.add(w)
    for i in range(len(word_list)-1,-1,-1):
        if(word_list[i][0]=="DELETE"):
            del word_list[i]
    for i in range(len(word_list)):
        del word_list[i][1]
    return word_list
def rank(word):
    for i in range(len(model.wv.index2word)):
        if model.wv.index2word[i]==word:
            return i+1
# Called when a request is received
def keyword_stem(w1):
    ps = PorterStemmer() 
    if " " in w1:
        if (ps.stem(w1.split()[0]) in keyword_mem) or (ps.stem(w1.split()[1]) in keyword_mem):
            return False
        else:
            keyword_mem.add(ps.stem(w1.split()[0]))
            keyword_mem.add(ps.stem(w1.split()[1]))
            return True
    else:    
        if ps.stem(w1) in keyword_mem:
            return False
        else:
            keyword_mem.add(ps.stem(w1))
            return True

def run(raw_data):
    # Get the input data as a numpy array
    data = list(json.loads(raw_data)['data'])
    # Get a prediction from the model
    keyword_mem.clear()
    listed_result=[]
    for w in data:
        mem.clear()
        if(keyword_stem(w)==False):
            continue
        else:
            try:
                list1=list(model.wv.most_similar(positive=w,topn=20, restrict_vocab=round(rank(w)*1.3)+2000))
            except:
                continue
            listed_result.append("Layer 1")
            listed_result.append("1")
            listed_result.append(w)
            list1=post_processing(w,list1)
            length = 0
            if len(list1)>5:
                length = 5
            else:
                length=len(list1)
            listed_result.append("Layer 2")
            listed_result.append(length)
            for i in range(length):
                listed_result.append(list1[i][0])
            listed_result.append("Layer 3")
            for i in range(length):
                list2=list(model.wv.most_similar(positive=list1[i][0],topn=20, restrict_vocab=round(rank(list1[i][0])*1.3)+2000))
                list2=post_processing(list1[i][0],list2)
                length2 = 0
                if len(list2)>2:
                    length2=2
                else:
                    lenght2=len(list2)
                listed_result.append(length2)
                for j in range(length2):
                    listed_result.append(list2[j][0])
                
    # Return the predictions as JSON
    return json.dumps(listed_result)

Overwriting chem_service/chem_predictions.py


In [22]:
from azureml.core.conda_dependencies import CondaDependencies 

# Add the dependencies for our model (AzureML defaults is already included)
myenv = CondaDependencies()
myenv.add_conda_package("gensim")
myenv.add_conda_package("nltk")



# Save the environment config as a .yml file
env_file = folder_name + "/chem_env.yml"
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

# Print the .yml file
with open(env_file,"r") as f:
    print(f.read())

Saved dependency info in chem_service/chem_env.yml
# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.
  - azureml-defaults

- gensim
- nltk
channels:
- anaconda
- conda-forge



In [23]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig

# Configure the scoring environment
inference_config = InferenceConfig(runtime= "python",
                                   source_directory = folder_name,
                                   entry_script="chem_predictions.py",
                                   conda_file="chem_env.yml")

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 2, memory_gb = 2)

service_name = "chem-service"

service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)

Running....................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [24]:
print(service.state)
print(service.get_logs())

# If you need to make a change and redeploy, you may need to delete unhealthy service using the following code:
#service.delete()

Healthy
2020-08-23T01:48:11,327085219+00:00 - gunicorn/run 
2020-08-23T01:48:11,327294020+00:00 - iot-server/run 
2020-08-23T01:48:11,328285227+00:00 - rsyslog/run 
2020-08-23T01:48:11,337234786+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_46ab9fb243ddb0b4a1a2b0acfbb24d4a/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_46ab9fb243ddb0b4a1a2b0acfbb24d4a/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_46ab9fb243ddb0b4a1a2b0acfbb24d4a/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_46ab9fb243ddb0b4a1a2b0acfbb24d4a/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_46ab9fb243ddb0b4a1a2b0acfbb24d4a/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)


In [25]:
for webservice_name in ws.webservices:
    print(webservice_name)

chem-service


In [26]:
import json

x_new = ["ether","hydrocarbon"]

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data (the web service will also accept the data in binary format)
predictions = service.run(input_data = input_json)

# Get the predicted class - it'll be the first (and only) one.
predicted_classes = json.loads(predictions)
print(predicted_classes)

['Layer 1', '1', 'ether', 'Layer 2', 5, 'alcohol', 'chloroform', 'soluble', 'benzene', 'acetone', 'Layer 3', 2, 'deliquescent', 'glycerol', 2, 'glacial', 'alkalies', 2, 'insoluble', 'slightly', 2, 'aromatic', 'aliphatic', 2, 'solvents', 'fungicide', 'Layer 1', '1', 'hydrocarbon', 'Layer 2', 5, 'aliphatic', 'aromatic', 'saturated', 'rings', 'aldehydes', 'Layer 3', 2, 'alkane', 'toluene', 2, 'lacquers', 'pungent', 2, 'disulfide', 'methane', 2, 'cyclic', 'hydrogen', 2, 'radical', 'synthesis known']


In [27]:
endpoint = service.scoring_uri
print(endpoint)

http://b719509c-a1a3-4565-947e-ef9c3480e1d7.canadacentral.azurecontainer.io/score


In [31]:
import requests
import json

x_new = ["ether"]

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})
print(input_json)
# Set the content type
headers = { 'Content-Type':'application/json' }

predictions = requests.post(endpoint, input_json, headers = headers)
predicted_classes = json.loads(predictions.json())


print (predicted_classes)

{"data": ["ether"]}
['Layer 1', '1', 'ether', 'Layer 2', 5, 'alcohol', 'chloroform', 'soluble', 'benzene', 'acetone', 'Layer 3', 2, 'deliquescent', 'glycerol', 2, 'glacial', 'alkalies', 2, 'insoluble', 'slightly', 2, 'aromatic', 'aliphatic', 2, 'solvents', 'fungicide']
